In [1]:
import argparse
from collections import defaultdict, deque
import cv2
import numpy as np
from ultralytics import YOLO
import supervision as sv

# Define SOURCE and TARGET constants
SOURCE = np.array(((800, 410), (1125, 410), (1920, 850), (0, 850)))

TARGET_WIDTH = 32
TARGET_HEIGHT = 140

TARGET = np.array(
    [
        [0, 0],
        [TARGET_WIDTH - 1, 0],
        [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
        [0, TARGET_HEIGHT - 1],
    ]
)

CLASS_NAMES = {
    2: "car",
    3: "motorcycle",
    5: "bus",
    7: "truck"
}

class ViewTransformer:
    def __init__(self, source: np.ndarray, target: np.ndarray) -> None:
        source = source.astype(np.float32)
        target = target.astype(np.float32)
        self.m = cv2.getPerspectiveTransform(source, target)

    def transform_points(self, points: np.ndarray) -> np.ndarray:
        if points.size == 0:
            return points
        reshaped_points = points.reshape(-1, 1, 2).astype(np.float32)
        transformed_points = cv2.perspectiveTransform(reshaped_points, self.m)
        return transformed_points.reshape(-1, 2)

if __name__ == "__main__":
    video_info = sv.VideoInfo.from_video_path(video_path='./asset/m6-motorway-trim.mp4')
    video_info.fps = 25
    
    model = YOLO("yolov8n.pt")

    byte_track = sv.ByteTrack(
        frame_rate=video_info.fps, track_activation_threshold=0.3
    )

    thickness = sv.calculate_optimal_line_thickness(
        resolution_wh=video_info.resolution_wh
    )
    text_scale = sv.calculate_optimal_text_scale(resolution_wh=video_info.resolution_wh)

    box_annotator = sv.BoxAnnotator(thickness=thickness)
    trace_annotator = sv.TraceAnnotator(
        thickness=thickness,
        trace_length=video_info.fps * 2,
        position=sv.Position.BOTTOM_CENTER,
    )

    # Two label annotators for top-left and bottom-center
    label_annotator_top_left = sv.LabelAnnotator(
        text_scale=text_scale,
        text_thickness=thickness,
        text_position=sv.Position.TOP_LEFT,
    )

    label_annotator_bottom = sv.LabelAnnotator(
        text_scale=text_scale,
        text_thickness=thickness,
        text_position=sv.Position.BOTTOM_CENTER,
    )

    frame_generator = sv.get_video_frames_generator(source_path='./asset/m6-motorway-trim.mp4')

    polygon_zone = sv.PolygonZone(polygon=SOURCE)
    view_transformer = ViewTransformer(source=SOURCE, target=TARGET)

    coordinates = defaultdict(lambda: deque(maxlen=video_info.fps))
    
    # Vehicle counts by type
    vehicle_counts = defaultdict(int)

    # Set to track unique tracker IDs
    processed_tracker_ids = set()

    with sv.VideoSink('./asset/m6-motorway-trim-result.mp4', video_info) as sink:
        with open('./asset/speed_results.txt', 'w') as f:  
            for frame in frame_generator:
                result = model(frame)[0]
                detections = sv.Detections.from_ultralytics(result)
                detections = detections[detections.confidence > 0.3]
                detections = detections[polygon_zone.trigger(detections)]
                detections = detections.with_nms(threshold=0.6)
                detections = byte_track.update_with_detections(detections=detections)

                points = detections.get_anchors_coordinates(
                    anchor=sv.Position.BOTTOM_CENTER
                )
                points = view_transformer.transform_points(points=points).astype(int)

                # Two label lists
                top_left_labels = []
                bottom_labels = []

                for tracker_id, [_, y], class_id in zip(detections.tracker_id, points, detections.class_id):
                    coordinates[tracker_id].append(y)
                    vehicle_type = CLASS_NAMES.get(class_id, "unknown")

                    top_left_labels.append(vehicle_type)  # Label for top-left

                    if vehicle_type != "unknown" and tracker_id not in processed_tracker_ids:
                        # Increment the count for this vehicle type only once per tracker_id
                        vehicle_counts[vehicle_type] += 1
                        processed_tracker_ids.add(tracker_id)  # Mark tracker_id as processed

                    if len(coordinates[tracker_id]) < video_info.fps / 2:
                        bottom_labels.append(f"#{tracker_id}")
                    else:
                        coordinate_start = coordinates[tracker_id][-1]
                        coordinate_end = coordinates[tracker_id][0]
                        distance = abs(coordinate_start - coordinate_end)
                        time = len(coordinates[tracker_id]) / video_info.fps
                        speed = distance / time * 3.6
                        bottom_labels.append(f"#{tracker_id} {int(speed)} km/h")
                        
                        f.write(f"Tracker ID: {tracker_id}, Type: {vehicle_type}, Speed: {int(speed)} km/h\n")

                annotated_frame = frame.copy()
                annotated_frame = trace_annotator.annotate(scene=annotated_frame, detections=detections)
                annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections)

                # Apply both label sets
                annotated_frame = label_annotator_top_left.annotate(
                    scene=annotated_frame, detections=detections, labels=top_left_labels
                )
                annotated_frame = label_annotator_bottom.annotate(
                    scene=annotated_frame, detections=detections, labels=bottom_labels
                )

                sink.write_frame(annotated_frame)
                cv2.imshow("frame", annotated_frame)
                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break

        print("\nVehicle Counts:")
        for vehicle, count in vehicle_counts.items():
            print(f"{vehicle}: {count}")

        cv2.destroyAllWindows()



0: 384x640 11 cars, 1 bus, 1 train, 2 trucks, 176.7ms
Speed: 11.3ms preprocess, 176.7ms inference, 18.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 bus, 1 train, 2 trucks, 123.9ms
Speed: 5.6ms preprocess, 123.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 bus, 1 train, 2 trucks, 66.0ms
Speed: 5.1ms preprocess, 66.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 1 bus, 1 train, 2 trucks, 62.5ms
Speed: 4.0ms preprocess, 62.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 bus, 1 train, 1 truck, 65.0ms
Speed: 2.0ms preprocess, 65.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 train, 2 trucks, 76.6ms
Speed: 2.0ms preprocess, 76.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 train, 1 truck, 56.4ms
Speed: 2.0ms preprocess, 56.4ms inference, 1.0ms p

In [12]:
import argparse
from collections import defaultdict, deque
import cv2
import numpy as np
from ultralytics import YOLO
import supervision as sv

# Define SOURCE and TARGET constants
SOURCE = np.array(((432,  24), (488, 709), (887, 592), (578,  17)))

TARGET_WIDTH = 5
TARGET_HEIGHT = 130

TARGET = np.array(
    [
        [0, 0],
        [TARGET_WIDTH - 1, 0],
        [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
        [0, TARGET_HEIGHT - 1],
    ]
)

CLASS_NAMES = {
    2: "car",
    3: "motorcycle",
    5: "bus",
    7: "truck"
}

class ViewTransformer:
    def __init__(self, source: np.ndarray, target: np.ndarray) -> None:
        source = source.astype(np.float32)
        target = target.astype(np.float32)
        self.m = cv2.getPerspectiveTransform(source, target)

    def transform_points(self, points: np.ndarray) -> np.ndarray:
        if points.size == 0:
            return points
        reshaped_points = points.reshape(-1, 1, 2).astype(np.float32)
        transformed_points = cv2.perspectiveTransform(reshaped_points, self.m)
        return transformed_points.reshape(-1, 2)

if __name__ == "__main__":
    video_info = sv.VideoInfo.from_video_path(video_path='./asset/videoplayback.mp4')
    video_info.fps = 25
    
    model = YOLO("yolo11n.pt")

    byte_track = sv.ByteTrack(
        frame_rate=video_info.fps, track_activation_threshold=0.3
    )

    thickness = sv.calculate_optimal_line_thickness(
        resolution_wh=video_info.resolution_wh
    )
    text_scale = sv.calculate_optimal_text_scale(resolution_wh=video_info.resolution_wh)

    box_annotator = sv.BoxAnnotator(thickness=thickness)
    trace_annotator = sv.TraceAnnotator(
        thickness=thickness,
        trace_length=video_info.fps * 2,
        position=sv.Position.BOTTOM_CENTER,
    )

    # Two label annotators for top-left and bottom-center
    label_annotator_top_left = sv.LabelAnnotator(
        text_scale=text_scale,
        text_thickness=thickness,
        text_position=sv.Position.TOP_LEFT,
    )

    label_annotator_bottom = sv.LabelAnnotator(
        text_scale=text_scale,
        text_thickness=thickness,
        text_position=sv.Position.BOTTOM_CENTER,
    )

    frame_generator = sv.get_video_frames_generator(source_path='./asset/videoplayback.mp4')

    polygon_zone = sv.PolygonZone(polygon=SOURCE)
    view_transformer = ViewTransformer(source=SOURCE, target=TARGET)

    coordinates = defaultdict(lambda: deque(maxlen=video_info.fps))
    
    # Vehicle counts by type
    vehicle_counts = defaultdict(int)

    # Set to track unique tracker IDs
    processed_tracker_ids = set()

    with sv.VideoSink('./asset/Merge-way-full.mp4-result.mp4', video_info) as sink:
        with open('./asset/speed_resultss.txt', 'w') as f:  
            for frame in frame_generator:
                result = model(frame)[0]
                detections = sv.Detections.from_ultralytics(result)
                detections = detections[detections.confidence > 0.3]
                detections = detections[polygon_zone.trigger(detections)]
                detections = detections.with_nms(threshold=0.6)
                detections = byte_track.update_with_detections(detections=detections)

                points = detections.get_anchors_coordinates(
                    anchor=sv.Position.BOTTOM_CENTER
                )
                points = view_transformer.transform_points(points=points).astype(int)

                # Two label lists
                top_left_labels = []
                bottom_labels = []

                for tracker_id, [_, y], class_id in zip(detections.tracker_id, points, detections.class_id):
                    coordinates[tracker_id].append(y)
                    vehicle_type = CLASS_NAMES.get(class_id, "unknown")

                    top_left_labels.append(vehicle_type)  # Label for top-left

                    if vehicle_type != "unknown" and tracker_id not in processed_tracker_ids:
                        # Increment the count for this vehicle type only once per tracker_id
                        vehicle_counts[vehicle_type] += 1
                        processed_tracker_ids.add(tracker_id)  # Mark tracker_id as processed

                    if len(coordinates[tracker_id]) < video_info.fps / 2:
                        bottom_labels.append(f"#{tracker_id}")
                    else:
                        coordinate_start = coordinates[tracker_id][-1]
                        coordinate_end = coordinates[tracker_id][0]
                        distance = abs(coordinate_start - coordinate_end)
                        time = len(coordinates[tracker_id]) / video_info.fps
                        speed = distance / time * 3.6
                        bottom_labels.append(f"#{tracker_id} {int(speed)} km/h")
                        
                        # Write to file only if tracker_id is new (not processed before)
                        f.write(f"Tracker ID: {tracker_id}, Type: {vehicle_type}, Speed: {int(speed)} km/h\n")

                annotated_frame = frame.copy()
                annotated_frame = trace_annotator.annotate(scene=annotated_frame, detections=detections)
                annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections)

                # Apply both label sets
                annotated_frame = label_annotator_top_left.annotate(
                    scene=annotated_frame, detections=detections, labels=top_left_labels
                )
                annotated_frame = label_annotator_bottom.annotate(
                    scene=annotated_frame, detections=detections, labels=bottom_labels
                )

                sink.write_frame(annotated_frame)
                cv2.imshow("frame", annotated_frame)
                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break

        print("\nVehicle Counts:")
        for vehicle, count in vehicle_counts.items():
            print(f"{vehicle}: {count}")

        cv2.destroyAllWindows()




0: 384x640 8 cars, 109.6ms
Speed: 7.5ms preprocess, 109.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 91.4ms
Speed: 3.0ms preprocess, 91.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 94.3ms
Speed: 2.0ms preprocess, 94.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 76.3ms
Speed: 6.3ms preprocess, 76.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 72.2ms
Speed: 1.0ms preprocess, 72.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 61.1ms
Speed: 2.0ms preprocess, 61.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 64.7ms
Speed: 1.0ms preprocess, 64.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 66.3ms
Speed: 2.0ms preprocess, 66.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 38

In [14]:
import argparse
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO
import supervision as sv
import csv

# Define SOURCE and TARGET constants
SOURCE = np.array(((432,  24), (488, 709), (887, 592), (578,  17)))

TARGET_WIDTH = 5
TARGET_HEIGHT = 130

TARGET = np.array(
    [
        [0, 0],
        [TARGET_WIDTH - 1, 0],
        [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
        [0, TARGET_HEIGHT - 1],
    ]
)

CLASS_NAMES = {
    2: "car",
    3: "motorcycle",
    5: "bus",
    7: "truck"
}

class ViewTransformer:
    def __init__(self, source: np.ndarray, target: np.ndarray) -> None:
        source = source.astype(np.float32)
        target = target.astype(np.float32)
        self.m = cv2.getPerspectiveTransform(source, target)

    def transform_points(self, points: np.ndarray) -> np.ndarray:
        if points.size == 0:
            return points
        reshaped_points = points.reshape(-1, 1, 2).astype(np.float32)
        transformed_points = cv2.perspectiveTransform(reshaped_points, self.m)
        return transformed_points.reshape(-1, 2)

if __name__ == "__main__":
    video_info = sv.VideoInfo.from_video_path(video_path='./asset/videoplayback.mp4')
    video_info.fps = 25

    model = YOLO("yolo11n.pt")

    byte_track = sv.ByteTrack(
        frame_rate=video_info.fps, track_activation_threshold=0.3
    )

    thickness = sv.calculate_optimal_line_thickness(
        resolution_wh=video_info.resolution_wh
    )
    text_scale = sv.calculate_optimal_text_scale(resolution_wh=video_info.resolution_wh)

    box_annotator = sv.BoxAnnotator(thickness=thickness)
    trace_annotator = sv.TraceAnnotator(
        thickness=thickness,
        trace_length=video_info.fps * 2,
        position=sv.Position.BOTTOM_CENTER,
    )

    label_annotator_top_left = sv.LabelAnnotator(
        text_scale=text_scale,
        text_thickness=thickness,
        text_position=sv.Position.TOP_LEFT,
    )

    label_annotator_bottom = sv.LabelAnnotator(
        text_scale=text_scale,
        text_thickness=thickness,
        text_position=sv.Position.BOTTOM_CENTER,
    )

    frame_generator = sv.get_video_frames_generator(source_path='./asset/videoplayback.mp4')

    polygon_zone = sv.PolygonZone(polygon=SOURCE)
    view_transformer = ViewTransformer(source=SOURCE, target=TARGET)

    vehicle_counts = defaultdict(int)
    processed_tracker_ids = set()

    with open('./asset/tracking_results.csv', mode='a', newline='') as csvfile:
        fieldnames = ["tracker_id", "vehicle_type"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        if csvfile.tell() == 0:
            writer.writeheader()

        try:
            with sv.VideoSink('./asset/TrackingOnlyResult.mp4', video_info) as sink:
                tracker_types = {}  # Moved outside the loop to persist vehicle types

                for frame in frame_generator:
                    result = model(frame)[0]
                    detections = sv.Detections.from_ultralytics(result)
                    detections = detections[detections.confidence > 0.5]
                    detections = detections[polygon_zone.trigger(detections)]
                    detections = detections.with_nms(threshold=0.6)
                    detections = byte_track.update_with_detections(detections=detections)

                    points = detections.get_anchors_coordinates(
                        anchor=sv.Position.BOTTOM_CENTER
                    )
                    points = view_transformer.transform_points(points=points).astype(int)

                    top_left_labels = []
                    bottom_labels = []

                    for tracker_id, point, class_id in zip(detections.tracker_id, points, detections.class_id):
                        # Removed coordinates tracking and speed calculation
                        if tracker_id not in tracker_types:
                            tracker_types[tracker_id] = CLASS_NAMES.get(class_id, "unknown")

                        vehicle_type = tracker_types[tracker_id]

                        top_left_labels.append(vehicle_type)
                        bottom_labels.append(f"#{tracker_id}")

                        if vehicle_type != "unknown" and tracker_id not in processed_tracker_ids:
                            vehicle_counts[vehicle_type] += 1
                            writer.writerow({"tracker_id": tracker_id, "vehicle_type": vehicle_type})
                            csvfile.flush()
                            processed_tracker_ids.add(tracker_id)

                    annotated_frame = trace_annotator.annotate(scene=frame.copy(), detections=detections)
                    annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections)

                    annotated_frame = label_annotator_top_left.annotate(
                        scene=annotated_frame, detections=detections, labels=top_left_labels
                    )
                    annotated_frame = label_annotator_bottom.annotate(
                        scene=annotated_frame, detections=detections, labels=bottom_labels
                    )

                    sink.write_frame(annotated_frame)
                    cv2.imshow("Tracking Only", annotated_frame)
                    if cv2.waitKey(1) & 0xFF == ord("q"):
                        break

        except Exception as e:
            print(f"Error: {e}")

        finally:
            csvfile.flush()
            print("Tracking data has been saved.")

        cv2.destroyAllWindows()
        print("Vehicle Counts:")
        for vehicle, count in vehicle_counts.items():
            print(f"{vehicle}: {count}")



0: 384x640 8 cars, 59.6ms
Speed: 2.0ms preprocess, 59.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 110.6ms
Speed: 3.0ms preprocess, 110.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 92.3ms
Speed: 5.0ms preprocess, 92.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 69.5ms
Speed: 2.0ms preprocess, 69.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 67.5ms
Speed: 2.0ms preprocess, 67.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 67.5ms
Speed: 1.1ms preprocess, 67.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 60.5ms
Speed: 3.0ms preprocess, 60.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 67.1ms
Speed: 2.0ms preprocess, 67.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 38

In [16]:
import argparse
from collections import defaultdict, deque
import cv2
import numpy as np
from ultralytics import YOLO
import supervision as sv
import csv

# Define SOURCE and TARGET constants
SOURCE = np.array(((422, 10), (535, 649), (801, 665), (594, 16)))

TARGET_WIDTH = 5
TARGET_HEIGHT = 130

TARGET = np.array(
    [
        [0, 0],
        [TARGET_WIDTH - 1, 0],
        [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
        [0, TARGET_HEIGHT - 1],
    ]
)

# Define Stop Zone (Before the giveaway sign)
STOP_ZONE = np.array(
    [
        (540, 307),
        (735, 310),
        (746, 557),
        (490, 555),
    ]
)

CLASS_NAMES = {
    2: "car",
    3: "motorcycle",
    5: "bus",
    7: "truck"
}

class ViewTransformer:
    def __init__(self, source: np.ndarray, target: np.ndarray) -> None:
        source = source.astype(np.float32)
        target = target.astype(np.float32)
        self.m = cv2.getPerspectiveTransform(source, target)

    def transform_points(self, points: np.ndarray) -> np.ndarray:
        if points.size == 0:
            return points
        reshaped_points = points.reshape(-1, 1, 2).astype(np.float32)
        transformed_points = cv2.perspectiveTransform(reshaped_points, self.m)
        return transformed_points.reshape(-1, 2)

if __name__ == "__main__":
    video_info = sv.VideoInfo.from_video_path(video_path='./asset/videoplayback.mp4')
    video_info.fps = 25

    model = YOLO("yolo11n.pt")

    byte_track = sv.ByteTrack(
        frame_rate=video_info.fps, track_activation_threshold=0.3
    )

    thickness = sv.calculate_optimal_line_thickness(
        resolution_wh=video_info.resolution_wh
    )
    text_scale = sv.calculate_optimal_text_scale(resolution_wh=video_info.resolution_wh)

    box_annotator = sv.BoxAnnotator(thickness=thickness)
    trace_annotator = sv.TraceAnnotator(
        thickness=thickness,
        trace_length=video_info.fps * 2,
        position=sv.Position.BOTTOM_CENTER,
    )

    label_annotator_top_left = sv.LabelAnnotator(
        text_scale=text_scale,
        text_thickness=thickness,
        text_position=sv.Position.TOP_LEFT,
    )

    label_annotator_bottom = sv.LabelAnnotator(
        text_scale=text_scale,
        text_thickness=thickness,
        text_position=sv.Position.BOTTOM_CENTER,
    )

    frame_generator = sv.get_video_frames_generator(source_path='./asset/videoplayback.mp4')

    polygon_zone = sv.PolygonZone(polygon=SOURCE)
    stop_zone = sv.PolygonZone(polygon=STOP_ZONE)  # Define the stop zone
    view_transformer = ViewTransformer(source=SOURCE, target=TARGET)

    vehicle_counts = defaultdict(int)
    processed_tracker_ids = set()
    stopped_vehicles = defaultdict(int)  # Track how long a vehicle has been stopped

    # Dictionary to store the latest status for each tracker ID
    tracker_status = {}

    with open('./asset/tracking_results.csv', mode='w', newline='') as csvfile:
        fieldnames = ["tracker_id", "vehicle_type", "status"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()  # Write the header once

        try:
            with sv.VideoSink('./asset/TrackingWithStopResult.mp4', video_info) as sink:
                tracker_types = {}  # Moved outside the loop to persist vehicle types

                for frame in frame_generator:
                    result = model(frame)[0]
                    detections = sv.Detections.from_ultralytics(result)
                    detections = detections[detections.confidence > 0.4]
                    detections = detections[polygon_zone.trigger(detections)]
                    detections = detections.with_nms(threshold=0.6)
                    detections = byte_track.update_with_detections(detections=detections)

                    # Get both original and transformed anchor points
                    anchor_points = detections.get_anchors_coordinates(anchor=sv.Position.BOTTOM_CENTER)
                    transformed_points = view_transformer.transform_points(points=anchor_points).astype(int)

                    top_left_labels = []
                    bottom_labels = []

                    for tracker_id, orig_point, trans_point, class_id in zip(
                        detections.tracker_id, anchor_points, transformed_points, detections.class_id
                    ):
                        if tracker_id not in tracker_types:
                            tracker_types[tracker_id] = CLASS_NAMES.get(class_id, "unknown")

                        vehicle_type = tracker_types[tracker_id]
                        status = "moving"

                        # Check if this specific vehicle is inside the stop zone using original point
                        pt = tuple(map(float, orig_point))  # ensure it's (x, y) and float
                        if cv2.pointPolygonTest(stop_zone.polygon.astype(np.float32), pt, False) >= 0:

                            stopped_vehicles[tracker_id] += 1

                            if stopped_vehicles[tracker_id] > video_info.fps * 2:
                                status = "stopped"
                            elif stopped_vehicles[tracker_id] > video_info.fps:
                                status = "slower"
                        else:
                            stopped_vehicles[tracker_id] = 0

                        # Update tracker status in memory
                        if tracker_id not in tracker_status or (
                            tracker_status[tracker_id]["status"] != "stopped" and
                            (status == "stopped" or (status == "slower" and tracker_status[tracker_id]["status"] == "moving"))
                        ):
                            tracker_status[tracker_id] = {"vehicle_type": vehicle_type, "status": status}

                            # Overwrite the CSV file with updated tracker statuses
                            csvfile.seek(0)  # Move to the start of the file
                            csvfile.truncate()  # Clear the file
                            writer.writeheader()  # Re-write the header
                            for tid, data in tracker_status.items():
                                writer.writerow({"tracker_id": tid, "vehicle_type": data["vehicle_type"], "status": data["status"]})
                            csvfile.flush()

                        # Update labels
                        if status == "stopped":
                            top_left_labels.append(f"{vehicle_type} stopped")
                        elif status == "slower":
                            top_left_labels.append(f"{vehicle_type} slower")
                        else:
                            top_left_labels.append(vehicle_type)

                        bottom_labels.append(f"#{tracker_id}")

                    # Ensure labels match the number of detections
                    while len(top_left_labels) < len(detections):
                        top_left_labels.append("")
                    while len(bottom_labels) < len(detections):
                        bottom_labels.append("")

                    annotated_frame = trace_annotator.annotate(scene=frame.copy(), detections=detections)
                    annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections)

                    annotated_frame = label_annotator_top_left.annotate(
                        scene=annotated_frame, detections=detections, labels=top_left_labels
                    )
                    annotated_frame = label_annotator_bottom.annotate(
                        scene=annotated_frame, detections=detections, labels=bottom_labels
                    )

                    # Draw stop zone polygon (optional but useful)
                    cv2.polylines(annotated_frame, [STOP_ZONE], isClosed=True, color=(0, 255, 255), thickness=2)

                    sink.write_frame(annotated_frame)
                    cv2.imshow("Tracking with Stop", annotated_frame)
                    if cv2.waitKey(1) & 0xFF == ord("q"):
                        break

        except Exception as e:
            print(f"Error: {e}")

        finally:
            csvfile.flush()
            print("Tracking data has been saved.")

        cv2.destroyAllWindows()
        print("Vehicle Counts:")
        for vehicle, count in vehicle_counts.items():
            print(f"{vehicle}: {count}")



0: 384x640 8 cars, 76.3ms
Speed: 3.0ms preprocess, 76.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 124.2ms
Speed: 4.8ms preprocess, 124.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 107.4ms
Speed: 3.1ms preprocess, 107.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 77.9ms
Speed: 2.0ms preprocess, 77.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 72.6ms
Speed: 1.0ms preprocess, 72.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 79.4ms
Speed: 2.0ms preprocess, 79.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 76.1ms
Speed: 4.5ms preprocess, 76.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 80.3ms
Speed: 2.0ms preprocess, 80.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict

# Path to the saved speed data file
file_path = './asset/speed_resultss.txt'

# Dictionary to store speed data per vehicle type
vehicle_speeds = defaultdict(list)
vehicle_counts = defaultdict(int)

# Read speed data from the file
with open(file_path, 'r') as f:
    for line in f:
        parts = line.strip().split(',')
        if len(parts) == 3:
            vehicle_type = parts[1].split(":")[1].strip()
            speed = int(parts[2].split(":")[1].strip().split()[0])  # Extract speed value
            vehicle_speeds[vehicle_type].append(speed)
            vehicle_counts[vehicle_type] += 1  # Count vehicles per type

# Compute and print statistics
print("\nVehicle Type Counts:")
for vehicle, count in vehicle_counts.items():
    print(f"{vehicle}: {count}")

print("\nSpeed Statistics by Vehicle Type:")
for vehicle, speeds in vehicle_speeds.items():
    avg_speed = np.mean(speeds)
    max_speed = np.max(speeds)
    min_speed = np.min(speeds)
    print(f"{vehicle} -> Avg: {avg_speed:.2f} km/h, Max: {max_speed} km/h, Min: {min_speed} km/h")

# Plot Speed Distribution by Vehicle Type
plt.figure(figsize=(10, 5))
for vehicle, speeds in vehicle_speeds.items():
    plt.hist(speeds, bins=10, alpha=0.6, label=vehicle)

plt.xlabel('Speed (km/h)')
plt.ylabel('Frequency')
plt.title('Speed Distribution by Vehicle Type')
plt.legend()
plt.grid(True)
plt.show()
